# 02-Monte Carlos Tree Search (MCTS) Tutorial

In this tutorial, you will learn:

-  The basics theory of Monte Carlos Tree Search (MCTS)
-  A naive AI for tic-tac-toe
-  MCTS for tic-tac-toe
-  MCTS with UCB1 for tic-tac-toe
-  Intuition behind Upper Confidence Bounded 1 (UCB1) (Why it works!)
-  How to program a MCTS

We will build the knowledge upon the real application (Tic Tac Toe) to get intuitions about what, how, and why MCTS works well in chess plays and other AI applications.

## 1. Tic-Tac-Toe 

## 1.1 Introduction to Tic-Tac-Toe


Tic-Tac-Toe is a game that two players take turns playing on a three-by-three board. One player plays Xs and the other Os until one player wins by placing three marks in a row, horizontally, vertically, or diagonally.

Try this online game to get a feeling of Tic-Tac-Toe. [[link](https://playtictactoe.org/)]

<p align="center">
<img src="media/online-game.png" alt="drawing" width="300"/>
</p>

## 1.2 Tic-Tac-Toe in Python

In our github repository, there is a python script that implements tic-tac-toe in python. User can play with the designed AI player through command interface. The command interface is shown as follows:

<p align="center">
<img src="media/tic-tac-toe-console.png" alt="drawing" width="400"/>
</p>

- Numbers on the left are the `row` number.
- Numbers on the left are the `column` number.
- User chooses the move position by input `#row, #column`, meaning the corresponding row and column numbers of the desired move position. For example, "1, 2" means choosing the move position at Row#1 and Column#2

Let us run the following code to run tic-tac-toe in python:

In [3]:
# import local script
from human_play import human_play

n, width, height = 3,3,3 # line number that wins, board width, boad height
ai_type = "pure_mcts"
is_humanMoveFirst = True # set False if you want AI to play first
human_play(n, width, height, ai_type, is_humanMoveFirst=is_humanMoveFirst)

Player 1 with X
Player 2 with O

       0       1       2

   2   _       _       _    


   1   _       _       _    


   0   _       _       _    


Your move: 1,2
Player 1 with X
Player 2 with O

       0       1       2

   2   _       _       _    


   1   _       _       X    


   0   _       _       _    


Player 1 with X
Player 2 with O

       0       1       2

   2   _       _       _    


   1   _       O       X    


   0   _       _       _    


Your move: 1,1
invalid move
Your move: 2,1
Player 1 with X
Player 2 with O

       0       1       2

   2   _       X       _    


   1   _       O       X    


   0   _       _       _    


Player 1 with X
Player 2 with O

       0       1       2

   2   _       X       O    


   1   _       O       X    


   0   _       _       _    


Your move: 1,0
Player 1 with X
Player 2 with O

       0       1       2

   2   _       X       O    


   1   X       O       X    


   0   _       _       _    


Player 1 with X

In the above tic-tac-toe game, you played against a AI player with the Monte Carlo tree Search (MCTS). MCTS is a sophisticated AI that is widely used in game theory.

In this tutorial, we will try to develop a Player with MCTS that will achieve human-level AI for the Tic-Tac-Toe Game.

## 2. A Naive AI for solving Tic-Tac-Toe

Here we try to explain what is state. The state is the representation information to describe the current situation of the environment. In the case of tic-tac-toe, the state is the current situation of the board. The following picture is three states in the chess game.

<!--![dsfdf](media/tic-tac-toe-states.png)-->
<p align="center">
<img src="media/tic-tac-toe-states.png" alt="drawing" width="300"/>
</p>

Note that the white "X" or "O" indicates the current turn for two players.

Given a state, we have one or multiple choices of available actions corresponding to the state. Let us take the first state in the above picture as example. The available actions can be listed as following:
<!--
![dsfdf](media/tic-tac-toe-case1.png)-->
<p align="center">
<img src="media/tic-tac-toe-case1.png" alt="drawing" width="300"/>
</p>

The player can put the "O"s to position #1 to #8 as the above picture shows. 

Ok, now if you are the player, which action among the available actions given this state will be the best or the smartest move?

Perhaps the most natural thing that comes to your mind is to calculate the winning ratio of action given a particular state.
But how to calculate the winning ratio? Maybe one of the feasible ways is to do simulations to estimate and foresee the winning ratio of actions. 
Just like Doctor Strange!
<!--
![](media/doctor-strange.gif)
[<img src="media/doctor-strange.gif" width="250"/>](media/doctor-strange.gif)-->
<p align="center">
<img src="media/doctor-strange.gif" alt="drawing" width="300"/>
</p>

For example, given action 1, we can do hundreds of simulations to estimate the winning ratio. Eventually, you might calculate the winning ratios for states like this

<p align="center">
<img src="media/tic-tac-toe-winning-ratio2.png" alt="drawing" width="300"/>
</p>

The number in black in the above picture is the probability of winning ratio. Note that probabilities of all actions sum to 1. Based on the result, you can make a "smart" moving, the one with the highest probability. That is Action #5, with a probability of 0.89, in the above picture.


## 3. Monte Carlos Tree Search for solving Tic-Tac-Toe

So far, you might feel that the naive approach might be too unrealistic. You are right. One of the most significant drawbacks is that the simulations grow with the number of all state-action pairs in the game. We can calculate the number of all state-action pairs for tic-tac-toe as


<img src="https://render.githubusercontent.com/render/math?math=9*8*7...*1=9!=362880">

If you do 100 simulations for an action-state pair, you need to do 362880*100 simulations. That might be durable for some of the computers. But given a 10 by 10 board, the number of state-pair is factorial of 100, 9.332622e+157, which is definitely out of computational power. In fact, given an "n by n" board, the number of state-pair is <img src="https://render.githubusercontent.com/render/math?math=n^2!">

So are there some methods to create a more data-efficient AI? You know when professional players play a chess game, they will plan their moves a few steps ahead. In such a method, they can try to foresee the resultant future states which action might lead to. In AI theory, there is a similar method that is able to foresee the states. That is Monte Carlos Tree Search.

### 3.1 Basics of Tree

Let us try to foresee by planning the move.

<p align="center">
<img src="media/tree.png" alt="drawing" width="400"/>
</p>

In the above picture, we illustrate the planning using a tree structure. Since the tree is large, only a part of the entire tree is shown in the picture. To simplify the representation of the above picture, we use the following illustration, simply using nodes and edges.


<p align="center">
<img src="media/tree-representation.png" alt="drawing" width="400"/>
</p>

The node represents state and the edge represents available action. This allows us to analyze the resultant states of each action and its further consequence. Let us look deeper into the tree.

<p align="center">
<img src="media/tree-deep.png" alt="drawing" width="300"/>
</p>

As you might notice, each state has a blue number. The blue number means the winning probability of "O" given the current state (or the value in reinforcement learning). The above picture shows the deep tree with 3 step-look ahead. 

### 3.2 Process of MCTS 
#### 3.2.1 How to traverse the tree <span style="color:blue">(Selection)</span>

So the next question might be how to jump from the parent node to the child node. Since the blue number for each state means the winning ratio of Player "O", the rules of traverse is easily set as:

- Choose the available action with Maximum winning ratios of "O" at the turn of "O".
- Choose the available action with Minimum winning ratios of "O" at the turn of "X".

Such a rule is known as "MinMax", which is a famous algorithm in game theory.

<p align="center">
<img src="media/minmax.png" alt="drawing" width="300"/>
</p>

Why do we try to minimize the winning ratios of "O" at the turn of "X"? Because we try to make the player "X", the opponent of "O", be "smart" on his decision. An analogy I like to think about is that people will tend to play against the opponent is better or at least the same with their own for chess play to improve their skill of playing chess. In general, the "MinMax" algorithm or the rules of traverse can impose some constraints for the exploration that finds the optimal policy.

#### 3.2.2 How to expand or "grow" the tree <span style="color:blue">(Expansion)</span>

 After you traverse the tree using the "MinMax" algorithm, you will reach leaf nodes (the bottom of the tree). If you want to grow the tree to see a few more move ahead, you need to expand (or in other words "grow") the tree. It is easy to expand the tree as:
 
 - Add new nodes (representing new states) to the leaf nodes.
 - Initialize the nodes with initial values. In our case, just we initialize the value of the winning ratio to zero.

<p align="center">
<img src="media/tree-grow.png" alt="drawing" width="300"/>
</p>

[wiki link](https://zh.wikipedia.org/wiki/%E6%9E%81%E5%B0%8F%E5%8C%96%E6%9E%81%E5%A4%A7%E7%AE%97%E6%B3%95)

#### 3.2.3 Simulation<span style="color:blue"> (Rollout)</span>  

From the perspective of frequentists, the probability of winning ratio can be estimated using statistics, such as the Monte Carlo methods. Let us check the following picture

<p align="center">
<img src="media/probability-calculation.png" alt="drawing" width="800"/>
</p>

n is the number of game plays starting at the current state, t is the number of winning plays starting at the current state. The estimated probability of winning ratio is formulated as

<p align="center">
<img src="https://render.githubusercontent.com/render/math?math=\hat{p} = t/ n">
</p>

As the `n` grows larger, the estimated probability get more accurate. So we might need to do simulations as many as possible to get an accurate winning ratio. Next, we will explain how to increase the number of simulated plays.


 For MCTS, after the traverse and expansion process, we will reach the leaf node. We will do rollout (or simulation in other words) when we find n=0.

<p align="center">
<img src="media/rollout.png" alt="drawing" width="300"/>
</p>

In the rollout, the learned policy will play against a random policy, which might be the policy that gives uniform random moves. After the simulation, we record the result.

#### 3.2.4 Update Result<span style="color:blue">(Backpropagation)</span> 

After a rollout, we will update the tree based on the result of the rollout. 

<p align="center">
 <img src="media/backpropagation.png" alt="drawing" width="600"/>
</p>

As the above picture shows, we divide the cases into two:

- Case 1 (Player win): n and t are incremented by 1 for the rollout state, its parent state, and its ancestral states.
- Case 2 (Player lose): n is incremented by 1 for the rollout state, its parent state, and its ancestral states.

You might notice that the value is updated backward for the tree, which is similar to the backpropagation of gradient descent for neural networks.

#### 3.2.5 Summary: A full process of MCTS 

<p align="center">
 <img src="media/mcts-process.png" alt="drawing" width="1000"/>
</p>

The process of MCTS for one playout is set as the following sequences: 

- Selection -> Expasion -> Rollout -> Backpropagation. 

For MCTS, in order to approximate the probabilities of winning ratios accurately, we will try to execute playouts as many as possible.


## 4 Monte Carlos Tree Search with UCB1

### 4.1 Introduction to MCTS with UCB1
After you play with MCTS for a while, you will find out there is a problem with the original MCTS algorithm. Specifically, the "MinMax" algorithm will always choose actions with the maximum probability. At the beginning of playouts when the tree is small, the estimated probability might be not accurate due to its small sampling number n. This might lead to choosing sub-optimal actions, while other actions that might be optimal will be not selected. In the terminology of reinforcement learning, the sub-optimal actions are "exploited" too much, leaving other actions "un-explored".

"Exploitation and Exploration" is a well-known trade-off in reinforcement learning. One might need to balance the tradeoff in order to find an optimal policy. 

UCB1 (Upper Confidence Bound) is able to balance the "Exploitation" and "Exploration" for MCTS. The UCB1 value for a state can be formulated as

<p align="center">
<img src="https://render.githubusercontent.com/render/math?math=UCB = {\frac{t}{n}} %2B 2\sqrt{\frac{ln(N)}{n}}">
</p>

where N is the n value for its parent states.

"Minmax" algorithm with UCB1 can be formulated as

- Choose the available action with Maximum UCB for your turn.
- Choose the available action with Minimum UCB for the opponent's turn.

For the sake of computational convenience, we maximize the negative probability for the opponent's turns. The "Minmax" algorithm with UCB1 can be reformulated as

- <img src="https://render.githubusercontent.com/render/math?math=UCB = {\frac{t}{n}} * (-1)^{k%2B1} %2B 2\sqrt{\frac{ln(N)}{n}}">
- Choose the available action with Maximum UCB.

Where K is the number of turns, here we assume that the first turn is your turn. 

### 4.2 Intuition of UCB1 

Why does UCB1 work for balancing the trade-off of "exploration" and "exploitation"?

Let look at the following cases:

-Case 1:

<p align="center">
 <img src="media/ucb-case1.png" alt="drawing" width="400"/>
</p>

At the bottom, there are two leaf nodes that have been visited since n is not equal to zero, While the other leaf nodes remain un-visited. You will find that UCB values for un-visited leaf nodes tend to infinity. Following the rule of "MinMax", those nodes that have never been visited before will be selected. In this case, the UCB values will encourage "exploration".


-Case 2:
<p align="center">
 <img src="media/ucb-case2.png" alt="drawing" width="400"/>
</p>

When all the sibling node has been visited many time, the difference among their the terms in UCB <img src="https://render.githubusercontent.com/render/math?math=2\sqrt{\frac{ln(N)}{n}}"> will be small. However, UCB will weight more on the difference of the other term <img src="https://render.githubusercontent.com/render/math?math={\frac{t}{n}} * (-1)^{k%2B1}">. In this case, the UCB values will encourage "exploitation".

In conclusion, UCB will encourage "Exploration" when the number of visits, n, for sibling nodes is small while encouraging "Exploitation" when n is large. This makes sense on some level since, at the beginning of playouts, the estimation of the winning ratio is not too accurate. Encouraging "Exploitation" will prevent the cases that miss out on the nodes leading to the optimal policy. When n is large, the estimation becomes more accurate. So encouraging "Exploitation" will make the tree grow deeper, which contributes to the data efficiency of finding the optimal solution.

## 5 Program a MCTS

In the repository, we have a file called [mcts_pure.py](https://github.com/linhongbin-ws/MAEG-3080-project/blob/master/mcts_pure.py), which implemented the MCTS AI player. Let us walk through the code.

### 5.1 Class `TreeNode()`

Since the node of tree has some similar properties, we can use a class to program it. Here we explain the attributes method and variables for the tree node class. 

---

We will start with the **init** function:

```py
class TreeNode(object):
    """A node in the MCTS tree. Each node keeps track of its own value Q,
    prior probability P, and its visit-count-adjusted prior score u.
    """

    def __init__(self, parent, prior_p):
        self._parent = parent
        self._children = {}  # a map from action to TreeNode
        self._n_visits = 0
        self._Q = 0
        self._u = 0
        self._P = prior_p
```
- `self._parent`: The address of parent treeNode, `None` if it is a root.
- `self._children = {}`: A List of addresses of children treeNode.
- `self._n_visits`: The number of visits for current node (`n` in this tutorial)
- `self._Q`: The winning ratio:
 
<img style="margin:0px 0 0 100px" src="https://render.githubusercontent.com/render/math?math={\frac{t}{n}} * (-1)^{k%2B1} ">

- `self._u`: the second term in the UCB equation:

<img style="margin:0px 0 0 100px" src="https://render.githubusercontent.com/render/math?math=2\sqrt{\frac{ln(N)}{n}}  ">

- `self._P`: the prior winning ratio of this node.


```py
    def get_value(self, c_puct):
        """Calculate and return the value for this node.
        It is a combination of leaf evaluations Q, and this node's prior
        adjusted for its visit count, u.
        c_puct: a number in (0, inf) controlling the relative impact of
            value Q, and prior probability P, on this node's score.
        """
        self._u = (c_puct * self._P *
                   np.sqrt(self._parent._n_visits) / (1 + self._n_visits))
        return self._Q + self._u

    def is_leaf(self):
        """Check if leaf node (i.e. no nodes below this have been expanded).
        """
        return self._children == {}

    def is_root(self):
        return self._parent is None
```

---

There are also some **elementary functions** for Treenode:

```py
    def get_value(self, c_puct):
        """Calculate and return the value for this node.
        It is a combination of leaf evaluations Q, and this node's prior
        adjusted for its visit count, u.
        c_puct: a number in (0, inf) controlling the relative impact of
            value Q, and prior probability P, on this node's score.
        """
        self._u = (c_puct * self._P *
                   np.sqrt(self._parent._n_visits) / (1 + self._n_visits))
        return self._Q + self._u

    def is_leaf(self):
        """Check if leaf node (i.e. no nodes below this have been expanded).
        """
        return self._children == {}

    def is_root(self):
        return self._parent is None
```

- `get_value`: Get UCB value, c_puct is a weight parameter, set to 2 for the equation 
<img style="margin:0px 0 0 100px" src="https://render.githubusercontent.com/render/math?math=2\sqrt{\frac{ln(N)}{n}}  ">
- `is_leaf`: Check if the node is the leaf node
- `is_root`: Check if the node is root node

### 5.2 Selection
```py
def select(self, c_puct):
    """Select action among children that gives maximum action value Q
    plus bonus u(P).
    Return: A tuple of (action, next_node)
    """
    return max(self._children.items(),
               key=lambda act_node: act_node[1].get_value(c_puct))
``` 

`select` function will select the children node with maximum UCB value among all childeren nodes.

### 5.3 Expansion
```py
def expand(self, action_priors):
    """Expand tree by creating new children.
    action_priors: a list of tuples of actions and their prior probability
        according to the policy function.
    """
    for action, prob in action_priors:
        if action not in self._children:
            self._children[action] = TreeNode(self, prob)
```

`expand` function will create new children nodes corresponding to available actions given the state of current node. Arguement `action_priors` is a list of tupes: (`action`, `prob`), where `action` is the Action No. and `prob` is the prior probability.

### 5.4 Backpropagation

```py
def update(self, leaf_value):
    """Update node values from leaf evaluation.
    leaf_value: the value of subtree evaluation from the current player's
        perspective.
    """
    # Count visit.
    self._n_visits += 1
    # Update Q, a running average of values for all visits.
    self._Q += 1.0*(leaf_value - self._Q) / self._n_visits

def update_recursive(self, leaf_value):
    """Like a call to update(), but applied recursively for all ancestors.
    """
    # If it is not root, this node's parent should be updated first.
    if self._parent:
        self._parent.update_recursive(-leaf_value)
    self.update(leaf_value)
```

A recursive function is implemented in the above code. This [link](https://techterms.com/definition/recursive_function) and [wiki](https://en.wikipedia.org/wiki/Recursion_(computer_science)) explain what the recursive function is.

- `update_recursive()` will update the current node and its parent node in a recursive manner, `leaf_value` is the value for updating
- `update()` will update the node value.
```py
self._Q += 1.0*(leaf_value - self._Q) / self._n_visits
```
will update self._Q by averaging all its historical values.


### 5.2 Class `MCTS()`

After we build the class for treeNode, we can create a class to implement MCTS.

#### 5.2.1 `Init()` 

```py
    def __init__(self, policy_value_fn, c_puct=5, n_playout=10000):
        """
        policy_value_fn: a function that takes in a board state and outputs
            a list of (action, probability) tuples and also a score in [-1, 1]
            (i.e. the expected value of the end game score from the current
            player's perspective) for the current player.
        c_puct: a number in (0, inf) that controls how quickly exploration
            converges to the maximum-value policy. A higher value means
            relying on the prior more.
        """
        self._root = TreeNode(None, 1.0)
        self._policy = policy_value_fn
        self._c_puct = c_puct
        self._n_playout = n_playout

```

- `c_puct`: a user param to control exploration
- `n_playout`: number of playouts
- `policy_value_fn`: A prior policy function. In this code we set a prior policy with uniform distribution

In the code, a prior policy with uniform distribution is programmed as:

```py
def policy_value_fn(board):
    """a function that takes in a state and outputs a list of (action, probability)
    tuples and a score for the state"""
    # return uniform probabilities and 0 score for pure MCTS
    action_probs = np.ones(len(board.availables))/len(board.availables)
    return zip(board.availables, action_probs), 0
```

### 5.2.2 Rollout

Inside the process of MCTS for each playout, rollout is executed. Here are the code of rollout:
```py
    def _evaluate_rollout(self, state, limit=1000):
        """Use the rollout policy to play until the end of the game,
        returning +1 if the current player wins, -1 if the opponent wins,
        and 0 if it is a tie.
        """
        player = state.get_current_player()
        for i in range(limit):
            end, winner = state.game_end()
            if end:
                break
            action_probs = rollout_policy_fn(state)
            max_action = max(action_probs, key=itemgetter(1))[0]
            state.do_move(max_action)
        else:
            # If no break from the loop, issue a warning.
            print("WARNING: rollout reached move limit")
        if winner == -1:  # tie
            return 0
        else:
            return 1 if winner == player else -1
```

In the rollout, the program will simluate 1000 rounds of tic-tac-toe given a specific state. Rollout policy will be used for self-playing. In the pure MCTS, the code will use the rollout policy with random move:

```py
def rollout_policy_fn(board):
    """a coarse, fast version of policy_fn used in the rollout phase."""
    # rollout randomly
    action_probs = np.random.rand(len(board.availables))
    return zip(board.availables, action_probs)
```


### 5.2.3 Playout
```py
    def _playout(self, state):
        """Run a single playout from the root to the leaf, getting a value at
        the leaf and propagating it back through its parents.
        State is modified in-place, so a copy must be provided.
        """
        node = self._root
        while(1):
            if node.is_leaf():

                break
            # Greedily select next move.
            action, node = node.select(self._c_puct)
            state.do_move(action)

        action_probs, _ = self._policy(state)
        # Check for end of game
        end, winner = state.game_end()
        if not end:
            node.expand(action_probs)
        # Evaluate the leaf node by random rollout
        leaf_value = self._evaluate_rollout(state)
        # Update value and visit count of nodes in this traversal.
        node.update_recursive(-leaf_value)
```

This part of code will execute the process of MCTS for each playout. The process of MCTS for one playout is set as the following sequences: 

- Selection -> Expasion -> Rollout -> Backpropagation. 


<p align="center">
 <img src="media/mcts-process.png" alt="drawing" width="1000"/>
</p>



### 5.2.4 `get_move()`

```py
def get_move(self, state):
    """Runs all playouts sequentially and returns the most visited action.
    state: the current game state
    Return: the selected action
    """
    for n in range(self._n_playout):
        state_copy = copy.deepcopy(state)
        self._playout(state_copy)
    return max(self._root._children.items(),
               key=lambda act_node: act_node[1]._n_visits)[0]
```
This function is a high-level function that compute the move with the highest probability of winning ratio. By default, we will run 2000 playouts to get the most accurate estimation.

### 5.2.5 `update_with_move()`
```py
def update_with_move(self, last_move):
    """Step forward in the tree, keeping everything we already know
    about the subtree.
    """
    if last_move in self._root._children:
        self._root = self._root._children[last_move]
        self._root._parent = None
    else:
        self._root = TreeNode(None, 1.0)
```

This function will step into the next move while keeping the structure of the sub-tree starting from the next move. For instance:

<p align="center">
<img src="media/step-into-tree.png" alt="drawing" width="600"/>
</p>


## 5.3 AI Player using MCTS algorithm

```py
class MCTSPlayer(object):
    """AI player based on MCTS"""
    def __init__(self, c_puct=5, n_playout=2000):
        self.mcts = MCTS(policy_value_fn, c_puct, n_playout)

    def set_player_ind(self, p):
        self.player = p

    def reset_player(self):
        self.mcts.update_with_move(-1)

    def get_action(self, board):
        sensible_moves = board.availables
        if len(sensible_moves) > 0:
            move = self.mcts.get_move(board)
            self.mcts.update_with_move(-1)
            return move
        else:
            print("WARNING: the board is full")
```

- `__init__()`: create a MCTS class
- `set_player_ind()`: set the ID of Player
- `reset_player()`: reset MCST by discarding all tree stucture.
- `get_action`: get the action with hightest winning ratio using MCTS. Tree will be reset by calling `self.mcts.update_with_move(-1)`

## 6 Homework (Practice)

1. play tic-tac-toe in your python console

```py
# import local script
from human_play import human_play

n, width, height = 3,3,3 # line number that wins, board width, boad height
ai_type = "pure_mcts"
is_humanMoveFirst = True # set False if you want AI to play first
human_play(n, width, height, ai_type, is_humanMoveFirst=is_humanMoveFirst)
```

2. reivew the codes: 

- [mcts_pure.py](https://github.com/linhongbin-ws/MAEG-3080-project/blob/master/mcts_pure.py)
- [human_play.py](https://github.com/linhongbin-ws/MAEG-3080-project/blob/master/human_play.py)


**Reference**:

1. Clear Explanation by John Levine: [link](https://www.youtube.com/watch?v=UXW2yZndl7U&t=2s&ab_channel=JohnLevine)

1. Chinese Course on MCTS: [link](https://www.youtube.com/watch?v=niIaKaWIRX0&ab_channel=%E4%B8%AD%E5%9B%BD%E5%A4%A7%E5%AD%A6MOOC-%E6%85%95%E8%AF%BE)

1. Survey Paper for MCTS (Deep Understanding on MCTS and its variations): [link](https://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=6145622&casa_token=AecTrST5MJYAAAAA:1UepYH0lA9-jdodOaItjidj0ie8kcKFAH65qh4F3AzkX1wiWrfNj4lb5Um-w7RJChEu0heo3&tag=1)

1. Simple code of MCTS [link](https://github.com/haroldsultan/MCTS)
